In [28]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow .python.framework import ops
ops.reset_default_graph()

In [29]:
import pandas as pd

In [30]:
# import some data to play with
df = pd.read_csv('iris_training.csv')
df1 = pd.read_csv('iris_test.csv')

In [31]:
#create features and labels
X = np.array(df.iloc[:,:4]) 
Y = np.array(df.iloc[:,-1])

In [32]:
#Y = pd.DataFrame(Y)

X_test = np.array(df1.iloc[:,:4]) 
Y_test = np.array(df1.iloc[:,-1])
#Y_test = pd.DataFrame(Y_test)

In [33]:
# Define the learning rate， batch_size etc.
learning_rate = 0.01
batch_size = 5
iter_num = 1500

In [34]:
# Define the normalized function
def min_max_normalized(data):
    c_m = np.max(data, axis=0)
    c_n = np.min(data, axis=0)
    return np.divide(data - c_m, c_m - c_n)

In [35]:
# Normalized processing, must be placed after the data set segmentation, 
# otherwise the test set will be affected by the training set
train_X = min_max_normalized(X)
test_X = min_max_normalized(X_test)

In [36]:
# Define placeholders
d = tf.placeholder(dtype=tf.float32, shape=[None, 4])
t = tf.placeholder(dtype=tf.float32, shape=[None, 1])

In [37]:
# Begin building the model framework
# Declare the variables that need to be learned and initialization
# There are 4 features here, A's dimension is (4, 1)
a = tf.Variable(tf.random_normal(shape=[4, 1]))
b = tf.Variable(tf.random_normal(shape=[1, 1]))
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [38]:
# Declare the model you need to learn
mod = tf.matmul(d, a) + b
# Add summary ops to collect data
w_h = tf.summary.histogram("weights", a)
b_h = tf.summary.histogram("biases", b)

In [41]:

cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=mod, labels=t))
# Define the optimizer
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
tf.summary.scalar("cost_function", cross_entropy)
# The default threshold is 0.5, rounded off directly
prediction = tf.round(tf.sigmoid(mod))
# Bool into float32 type
correct = tf.cast(tf.equal(prediction, t), dtype=tf.float32)
# Average
accuracy = tf.reduce_mean(correct)
tf.summary.scalar("Accuracy", accuracy)
# End of the definition of the model framework

<tf.Tensor 'Accuracy:0' shape=() dtype=string>

In [43]:
# Start training model
# Define the variable that stores the result
loss_t = []
trainining_acc = []
testing_acc = []
# Add summary ops to collect data
# Initializing the variables
init = tf.global_variables_initializer()

# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    
    # Set the logs writer to the folder /tmp/tensorflow_logs
    summary_writer = tf.summary.FileWriter('D:\\Users\\703222761\\Documents\\assign\\new term\\dl\\ques2', graph=sess.graph)
    
 # training model
    for epoch in range(iter_num):
        
    # Generate random batch index
        b_i = np.random.choice(len(train_X), size=batch_size)
        total_batch = int(len(train_X)/batch_size)
        batch_t_X = train_X[b_i]
        batch_t_y = np.matrix(Y[b_i]).T
        sess.run(train_step, feed_dict={d: batch_t_X, t: batch_t_y})
        temp_loss = sess.run(cross_entropy, feed_dict={d: batch_t_X, t: batch_t_y})
        summary_str = sess.run(merged_summary_op, feed_dict={d: batch_t_X, t: batch_t_y})
        summary_writer.add_summary(summary_str, epoch*total_batch)
    # convert into a matrix, and the shape of the placeholder to correspond
        temp_train_acc = sess.run(accuracy, feed_dict={d: train_X, t: np.matrix(Y).T})
        temp_test_acc = sess.run(accuracy, feed_dict={d: test_X, t: np.matrix(Y_test).T})
    # recode the result
        loss_t.append(temp_loss)
        trainining_acc.append(temp_train_acc)
        testing_acc.append(temp_test_acc)
    # output
    if (epoch + 1) % 100 == 0:
        print('epoch: {:4d} loss: {:5f} train_acc: {:5f} test_acc: {:5f}'.format(epoch + 1, temp_loss,
                                                                          temp_train_acc, temp_test_acc))
        


epoch: 1500 loss: -0.206602 train_acc: 0.350000 test_acc: 0.266667


In [ ]:
###Question 5

In [65]:
num_epochs = 1500

In [66]:
df1.columns

Index(['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width',
       'Species'],
      dtype='object')

In [67]:
feature_columns = df.iloc[:,:-1]

In [68]:
labels = df.iloc[:,4:]

In [69]:
# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[5, 10, 5],
                                            n_classes=3,
                                            model_dir="/tmp/iris_model")

Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D591293400>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5

In [70]:
# Fit model.
classifier.fit(x=X,
               y=Y,
               steps=2000)


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to the Estimator interface.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please access pa

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x000001D599B746A0>, 'hidden_units': [5, 10, 5], 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x000001D5939452F0>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [132]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(x=X_test,
                                     y=Y_test)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Starting evaluation at 2019-01-18-13:29:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-18-13:29:42
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.96666664, global_

In [ ]:
#questoin 3 part B